In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree,ensemble,metrics

from rule import Rule
from rule_extraction import rule_extract,draw_tree,rules_vote

import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys
from datetime import datetime

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def getFitness(individual, X, y):
    """
    Feature subset fitness function
    """

    if(individual.count(0) != len(individual)):
        # get index with value 0
        cols = [index for index in range(
            len(individual)) if individual[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = LogisticRegression()

        return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
    else:
        return(0,)


def geneticAlgorithm(X, y, n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness, X=X, y=y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof


def bestIndividual(hof, X, y):
    """
    Get the best individual
    """
    maxAccurcy = 0.0
    for individual in hof:
    	print(individual.fitness.values)
    	print(maxAccurcy)
    	if(individual.fitness.values[0] > maxAccurcy):
            maxAccurcy = individual.fitness.values
            _individual = individual

    _individualHeader = [list(X)[i] for i in range(
        len(_individual)) if _individual[i] == 1]
    return _individual.fitness.values, _individual, _individualHeader


# def getArguments():
#     """
#     Get argumments from command-line
#     If pass only dataframe path, pop and gen will be default
#     """
#     dfPath = sys.argv[1]
#     if(len(sys.argv) == 4):
#         pop = int(sys.argv[2])
#         gen = int(sys.argv[3])
#     else:
#         pop = 10
#         gen = 2
#     return dfPath, pop, gen


def gafeature():
    # get dataframe path, population number and generation number from command-line argument
    n_pop = 20
    n_gen = 6
    # read dataframe from csv
    df = pd.read_csv('mental.csv')

    # encode labels column to numbers
    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1])
    X = df.iloc[:, :-1]

    # get accuracy with all features
    individual = [1 for i in range(len(X.columns))]
    print("Accuracy with all features: \t" + str(getFitness(individual, X, y)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(X, y, n_pop, n_gen)

    # select the best individual
    accuracy, individual, header = bestIndividual(hof, X, y)
    print('Best Accuracy: \t' + str(accuracy))
    print('Number of Features in Subset: \t' + str(individual.count(1)))
    print('Individual: \t\t' + str(individual))
    print('Feature Subset\t: ' + str(header))

    return header

In [3]:
gafeature()

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\lin

Accuracy with all features: 	(0.9450965858516696,)



C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

...

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anac

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

6  	13    	0.963788	0.960192	0.968958
(0.9689577043976222,)
0.0
(0.9673861628956459,)
(0.9689577043976222,)
(0.9673671905636301,)
(0.9689577043976222,)
(0.9665545921384332,)
(0.9689577043976222,)
(0.9658018735800317,)
(0.9689577043976222,)
(0.964982901248016,)
(0.9689577043976222,)
(0.9642334443372444,)
(0.9689577043976222,)
(0.964183050636457,)
(0.9689577043976222,)
(0.9641703028228189,)
(0.9689577043976222,)
(0.9641640783044412,)
(0.9689577043976222,)
(0.9634334443372443,)
(0.9689577043976222,)
(0.9634018735800318,)
(0.9689577043976222,)
(0.9634018735800318,)
(0.9689577043976222,)
(0.9633923874140239,)
(0.9689577043976222,)
(0.9626208459120476,)
(0.9689577043976222,)
(0.9626113597460396,)
(0.9689577043976222,)
(0.9626113597460396,)
(0.9689577043976222,)
(0.9626082474868507,)
(0.9689577043976222,)
(0.9626082474868507,)
(0.9689577043976222,)
(0.9625829012480158,)
(0.9689577043976222,)
(0.9625829012480158,)
(0.9689577043976222,)
(0.962576676729638,)
(0.9689577043976222,)
(0.961842930503

C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\HARSHITA\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\HARSHITA\anac

['Age',
 'mental_health_consequence',
 'coworkers',
 'obs_consequence',
 'mental health disorder in the past',
 'have a mental health disorder']

In [4]:
# fetch dataset
data = pd.read_csv('mental.csv', usecols = ['Age','family_history','remote_work','benefits','care_options','seek_help','leave','mental_health_consequence','supervisor','mental_health_interview','phys_health_interview','have a mental health disorder','diagnosed with a mental health condition by a medical professional'])

# drop NA records since most Tree algorithm cannot handle
data.dropna(inplace=True)

# split training/test sets
X_train, X_test, y_train, y_test = train_test_split(data[['Age','family_history','remote_work','benefits','care_options','seek_help','leave','mental_health_consequence','supervisor','mental_health_interview','phys_health_interview','have a mental health disorder','diagnosed with a mental health condition by a medical professional']], 
                                                    data.mental_health_consequence, test_size=0.2, random_state=0)

# dataset shape
print(X_train.shape, X_test.shape)
print(X_train.head(5))

(1005, 13) (252, 13)
     Age  family_history  remote_work  benefits  care_options  seek_help  \
159    7               1            0         0             1          1   
435   10               0            0         0             0          0   
554    9               1            1         1             0          1   
985   11               0            1         1             0          1   
933   20               1            0         1             1          1   

     leave  mental_health_consequence  supervisor  mental_health_interview  \
159      4                          0           0                        1   
435      0                          1           2                        1   
554      2                          0           1                        1   
985      4                          1           2                        1   
933      3                          2           1                        1   

     phys_health_interview  have a mental health diso

In [5]:
X_train_proceeded = X_train[['Age','family_history','remote_work','benefits','care_options','seek_help','leave','supervisor','mental_health_interview','phys_health_interview','have a mental health disorder','diagnosed with a mental health condition by a medical professional']]
X_test_proceeded = X_test[['Age','family_history','remote_work','benefits','care_options','seek_help','leave','supervisor','mental_health_interview','phys_health_interview','have a mental health disorder','diagnosed with a mental health condition by a medical professional']]
print(X_train_proceeded.head())

     Age  family_history  remote_work  benefits  care_options  seek_help  \
159    7               1            0         0             1          1   
435   10               0            0         0             0          0   
554    9               1            1         1             0          1   
985   11               0            1         1             0          1   
933   20               1            0         1             1          1   

     leave  supervisor  mental_health_interview  phys_health_interview  \
159      4           0                        1                      1   
435      0           2                        1                      1   
554      2           1                        1                      2   
985      4           2                        1                      2   
933      3           1                        1                      0   

     have a mental health disorder  \
159                              1   
435                   

In [6]:
model_RF_clf = ensemble.RandomForestClassifier(max_depth=4,n_estimators=3,random_state=9)
model_RF_clf.fit(X_train_proceeded,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False)

In [7]:
# model performance on test set
y_pred_test = model_RF_clf.predict(X_test_proceeded)
print(metrics.confusion_matrix(y_test,y_pred_test))

[[43 32 17]
 [14 82  5]
 [37  8 14]]


In [8]:
start_time = datetime.now()
rules,_ = rule_extract(model=model_RF_clf,feature_names=X_train_proceeded.columns)
for i in rules:
    print(i)
end_time = datetime.now()

seek_help > 1.5 and supervisor > 0.5 and supervisor <= 1.5 and family_history > 0.5
seek_help <= 1.5 and supervisor <= 1.5 and family_history <= 0.5 and leave > 2.5
seek_help > 1.5 and supervisor > 0.5 and supervisor > 1.5 and Age <= 4.5
seek_help > 1.5 and supervisor <= 0.5 and benefits > 1.5 and care_options <= 0.5
seek_help <= 1.5 and supervisor <= 1.5 and family_history <= 0.5 and leave <= 2.5
seek_help <= 1.5 and supervisor > 1.5 and Age <= 23.5 and Age <= 12.5
seek_help <= 1.5 and supervisor > 1.5 and Age <= 23.5 and Age > 12.5
seek_help > 1.5 and supervisor <= 0.5 and benefits <= 1.5
seek_help <= 1.5 and supervisor > 1.5 and Age > 23.5 and have a mental health disorder <= 1.5
seek_help > 1.5 and supervisor > 0.5 and supervisor <= 1.5 and family_history <= 0.5
seek_help <= 1.5 and supervisor <= 1.5 and family_history > 0.5 and supervisor <= 0.5
seek_help > 1.5 and supervisor <= 0.5 and benefits > 1.5 and care_options > 0.5
seek_help <= 1.5 and supervisor > 1.5 and Age > 23.5 and 

In [9]:
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.038417
